# TripScore — Live Recommendation Walkthrough

This notebook runs the **real** pipeline end-to-end with live ingestion:
- TDX samples (bus / YouBike / metro / optional parking)
- Open-Meteo weather
- Features + scoring + explainable recommendations

Prereqs:
- Network access
- `TDX_CLIENT_ID` and `TDX_CLIENT_SECRET` available (via `.env` or environment)


In [1]:
import os
import sys
from pathlib import Path
from json import load

def find_repo_root(start: Path) -> Path:
    p = start.resolve()
    for _ in range(10):
        if (p / "src").exists() and (p / "src" / "tripscore").exists():
            return p
        if (p / ".git").exists():
            return p
        if (p / ".env").exists():
            return p
        p = p.parent
    return start.resolve()

repo_root = find_repo_root(Path.cwd())
src_dir = repo_root / "src"
if src_dir.exists():
    sys.path.insert(0, str(src_dir))


print("repo_root:", repo_root)

try:
    from tripscore.core.env import load_dotenv_if_present, get_project_root

    env_path = load_dotenv_if_present()
    print("project_root:", get_project_root())
    print("loaded .env:", env_path)
except Exception as e:
    print("dotenv/root helper unavailable:", type(e).__name__, str(e))

print("TDX_CLIENT_ID configured:", bool(os.getenv("TDX_CLIENT_ID")))
print("TDX_CLIENT_SECRET configured:", bool(os.getenv("TDX_CLIENT_SECRET")))


repo_root: /home/justin/web-projects/tripscore
project_root: /home/justin/web-projects/tripscore
loaded .env: /home/justin/web-projects/tripscore/.env
TDX_CLIENT_ID configured: True
TDX_CLIENT_SECRET configured: True


In [2]:
from pathlib import Path

from tripscore.config.settings import get_settings
from tripscore.core.cache import FileCache
from tripscore.core.env import resolve_project_path
from tripscore.ingestion.tdx_client import TdxClient
from tripscore.ingestion.weather_client import WeatherClient

settings = get_settings()
cache_dir = resolve_project_path(settings.cache.dir)
cache = FileCache(cache_dir, enabled=settings.cache.enabled, default_ttl_seconds=settings.cache.default_ttl_seconds)

tdx = TdxClient(settings, cache)
weather = WeatherClient(settings, cache)

print("TDX city:", settings.ingestion.tdx.city)
print("Cache dir:", cache_dir)
print("Weather base_url:", settings.ingestion.weather.base_url)

TDX city: Taipei
Cache dir: /home/justin/web-projects/tripscore/.cache/tripscore
Weather base_url: https://api.open-meteo.com/v1/forecast


In [3]:
# Live ingestion sanity checks (sample fetches avoid large pagination / rate limits)

try:
    bus = tdx.get_bus_stops_sample(top=10)
    print("bus_stops_sample:", len(bus), "example:", bus[0])
except Exception as e:
    print("bus_stops_sample unavailable:", type(e).__name__, str(e))

try:
    bike = tdx.get_youbike_station_statuses_sample(top=10)
    print("youbike_sample:", len(bike), "example:", bike[0])
except Exception as e:
    print("youbike_sample unavailable:", type(e).__name__, str(e))

try:
    metro = tdx.get_metro_stations_sample(top=10)
    print("metro_sample:", len(metro), "example:", metro[0])
except Exception as e:
    print("metro_sample unavailable:", type(e).__name__, str(e))

# Parking availability differs by city; 404 can be normal.
try:
    parking = tdx.get_parking_lot_statuses_sample(top=10)
    print("parking_sample:", len(parking), "example:", parking[0] if parking else None)
except Exception as e:
    print("parking_sample unavailable (can be normal):", type(e).__name__, str(e))


bus_stops_sample: 10 example: BusStop(stop_uid='TPE10000', name='蘆洲總站', lat=25.089056, lon=121.465804)
youbike_sample: 10 example: BikeStationStatus(station_uid='TPE500101001', name='YouBike2.0_捷運科技大樓站', lat=25.02605, lon=121.5436, available_rent_bikes=4, available_return_bikes=24)
metro_sample: 10 example: MetroStation(station_uid='TRTC-BL01', name='頂埔', lat=24.959351, lon=121.418744, operator='TRTC')
parking_sample unavailable (can be normal): HTTPStatusError Client error '404 Not Found' for url 'https://tdx.transportdata.tw/api/basic/v2/Parking/OffStreet/ParkingLot/City/Taipei?%24format=JSON&%24top=10&%24skip=0&%24select=ParkingLotUID%2CParkingLotName%2CParkingLotPosition%2CTotalSpaces'
For more information check: https://httpstatuses.com/404


In [4]:
# Build a live query (edit origin/time window to match your use case)
from datetime import datetime
from zoneinfo import ZoneInfo

from tripscore.domain.models import GeoPoint, TimeWindow, UserPreferences

tz = ZoneInfo(settings.app.timezone)

# Taipei Main Station (example)
origin = GeoPoint(lat=25.0478, lon=121.5170)

# Choose a window; for real use, pick the next few hours.
start = datetime.now(tz=tz).replace(minute=0, second=0, microsecond=0)
end = start.replace(hour=min(start.hour + 6, 23))

prefs = UserPreferences(
    origin=origin,
    time_window=TimeWindow(start=start, end=end),
    max_results=10,
    tag_weights={"food": 1.0, "culture": 0.6, "indoor": 0.3},
)

prefs

UserPreferences(origin=GeoPoint(lat=25.0478, lon=121.517), time_window=TimeWindow(start=datetime.datetime(2026, 1, 18, 13, 0, tzinfo=zoneinfo.ZoneInfo(key='Asia/Taipei')), end=datetime.datetime(2026, 1, 18, 19, 0, tzinfo=zoneinfo.ZoneInfo(key='Asia/Taipei'))), preset=None, max_results=10, component_weights=None, weather_rain_importance=None, avoid_crowds_importance=None, family_friendly_importance=None, tag_weights={'food': 1.0, 'culture': 0.6, 'indoor': 0.3}, required_tags=[], excluded_tags=[], settings_overrides=None)

In [5]:
# Run live recommendations
from tripscore.recommender.recommend import recommend

result = recommend(
    prefs,
    settings=settings,
    tdx_client=tdx,
    weather_client=weather,
)

print("generated_at:", result.generated_at)
print("results:", len(result.results))
print("top 5:")
for i, item in enumerate(result.results[:5], start=1):
    dest = item.destination
    total = item.breakdown.total_score
    print(f"{i:>2}. {dest.name}  total={total:.3f}  tags={dest.tags[:5]}")


TDX request failed with status=429; retrying in 50.00s (attempt 1/6)


KeyboardInterrupt: 

In [6]:
# Inspect explainability for one result
idx = 0  # change this to inspect another destination
item = result.results[idx]

print("destination:", item.destination.name)
print("total:", round(item.breakdown.total_score, 3))

for comp in item.breakdown.components:
    print("\n-", comp.name)
    print("  score:", round(comp.score, 3), "weight:", round(comp.weight, 2), "contribution:", round(comp.contribution, 3))
    if comp.reasons:
        print("  reasons:", "; ".join(comp.reasons[:6]))
    # `comp.details` includes raw-ish fields and intermediate metrics for auditing.
    # Uncomment to inspect:
    # from pprint import pprint
    # pprint(comp.details)


NameError: name 'result' is not defined